In [1]:
import os
import sys
import logging
import datetime
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
sys.path.insert(1, r'methods')

from data_methods import getData, standardize
# from dimensionality_reduction import reduce_dimensionality,reduce_with
# from beamSearch import EMM, as_string
# from adjPysubgroup import adjustedBestFirstSearch, adjustedDFS, adjustedApriori
from qualityMeasures import calc_result_bs, calc_result_ps
from interpretabilityMeasures import Feature_Correlation_Scores, DBI_beam, DBI_ps

In [3]:
# Loading the object using dill
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\sd_results'
file_name = 'Ionosphere-data-reductions.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    dfs_dict =pickle.load(f)

In [4]:
# datasets = []
# for dataset in dataset:
#     file_name = '-data-reductions.pkl'
#     en_methods = ['vanilla', 'auto_encoder', 'PCA', 'SPCA']
#     for en_method in en_methods:
#         fcs, fcss = Feature_Correlation_Scores(loaded_dict['vanilla'], loaded_dict[en_method])
#         print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

In [15]:
en_methods = ['auto_encoder', 'PCA', 'SPCA']
for en_method in en_methods:
    fcs, fcss = Feature_Correlation_Scores(dfs_dict['vanilla'], dfs_dict[en_method])
    print(en_method, "    ", "\n FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))

auto_encoder      
 FCS:  0.59     FCSS 0.12
PCA      
 FCS:  0.65     FCSS 0.14
SPCA      
 FCS:  0.72     FCSS 0.19


In [3]:
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\results_renamed'
file_name = 'Soybean-results.pkl'
with open(os.path.join(sd_results_path,file_name), 'rb') as f:
    results_dict =pickle.load(f)

In [16]:
bfs = results_dict['vanilla']['Best-First Search']['results_org']
bfs_en = results_dict['PCA']['Best-First Search']['results_org']

In [17]:
vanilla = results_dict['auto_encoder']['Beam Search']['results_org']

In [18]:
subgroups_n = [i for i in vanilla.get_values()]

In [21]:
df = dfs_dict['auto_encoder']

In [22]:
df[df.eval(str(subgroups_n[1][1]).replace("', '", ' and ').replace("['", '').replace("']", ''))]

,target,cat1,cat2,cat3,cat4,cat5
36,0,2.670305,-0.637574,-0.008092,0.009340,1.814592
398,0,2.418383,-0.990442,-0.221028,0.080094,1.067282
513,0,3.184095,-1.293544,-0.379369,0.241604,2.213722
588,0,2.885142,-0.685566,-0.218985,0.358690,1.813227
632,0,-0.679030,-0.816299,-0.315998,0.162555,0.262455
...,...,...,...,...,...,...
8118,1,1.369584,-0.718166,1.165795,0.577131,-0.751188
8119,0,-0.775223,1.175854,-0.006725,2.146407,1.737964
8121,0,-1.062942,0.916443,0.504892,2.226397,2.688123
8122,1,1.088449,-0.264263,0.256847,1.536109,0.332227


In [47]:
str(subgroups_n[1][1])

"['cat2 > -0.8140837550163269', 'cat2 <= -0.17320257425308228']"

In [5]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
original_df = dfs_dict['vanilla']
features = [i for i in original_df.columns if i !='target']

In [6]:
OHenc = OneHotEncoder() 
dataCategorical = OHenc.fit_transform(original_df[features]).toarray()
original_encoded = pd.DataFrame(dataCategorical)

In [24]:
subgroups = results_dict['SPCA']['Depth-First Search']['results_org']
subgroups[subgroups['relative_size_sg'] >= 0.5]

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift
64,0.0,Dataset,420,420,183,183,0,1.0,0.0,1.0,0.0,0.435714,NaN,0.435714,1.0


In [23]:
sd_results_path = r'C:\Users\20193723\OneDrive - TU Eindhoven\Documents\Research Topics in Data Mining\Interpretable-Subgroup-Discovery\results_renamed'
dct_datasets = ['Arrhythmia'] # to test <- Remove later
categorical_datasets = ['Mushroom', 'Adult']
for dataset in dct_datasets:
    #Load dataframes and results
    df_file_name = dataset+'-data-reductions.pkl'
    results_file_name = dataset+'-results.pkl'
    with open(os.path.join(sd_results_path,df_file_name), 'rb') as f:
        dfs_dict =pickle.load(f)
    with open(os.path.join(sd_results_path,results_file_name), 'rb') as f:
        results_dict =pickle.load(f)
    #One-hot encode vanilla dataframe if categorical
    original_df = dfs_dict['vanilla']
    if dataset in categorical_datasets:
       features = [i for i in original_df.columns if i !='target']
       OHenc = OneHotEncoder() 
       dataCategorical = OHenc.fit_transform(original_df[features]).toarray()
       original_df = pd.DataFrame(dataCategorical)
    #Compute and print scores and index
    print(dataset, ":    \n")
    en_methods = ['auto_encoder', 'PCA', 'SPCA']
    ps_sd_methods = ['Apriori', 'Best-First Search', 'Depth-First Search']
    for en_method in en_methods:
        fcs, fcss = Feature_Correlation_Scores(original_df, dfs_dict[en_method])
        print(en_method, ":    ", "\n   FCS: ", round(fcs, 2), "    FCSS", round(fcss, 2))
        for sd_method in ps_sd_methods:
            dbi = DBI_ps(results_dict[en_method][sd_method]['results_org'], dfs_dict[en_method])
            print("DBI "+sd_method+": ", dbi)
        subgroups_beam = [i for i in results_dict[en_method]['Beam Search']['results_org'].get_values()]
        dbi = DBI_beam(subgroups_beam, dfs_dict[en_method])
        print("DBI Beam Search: ", dbi, "\n")
    print("\n\n")

Arrhythmia :    

auto_encoder :     
   FCS:  0.7     FCSS 0.31
DBI Apriori:  176.04204264686868
DBI Best-First Search:  185.5963747531536
DBI Depth-First Search:  184.90905777288822
DBI Beam Search:  1.2806238656859177e+18 

PCA :     
   FCS:  0.76     FCSS 0.25
DBI Apriori:  9.128979268086724
DBI Best-First Search:  8.782523375745633


UndefinedVariableError: name 'Dataset' is not defined

['attribute0', 'attribute1', 'attribute2', 'attribute3', 'attribute4', 'attribute5', 'attribute6', 'attribute7', 'attribute8', 'attribute9', 'attribute10', 'attribute11', 'attribute12', 'attribute14', 'attribute15', 'attribute16', 'attribute17', 'attribute18', 'attribute20', 'attribute21', 'attribute22', 'attribute23', 'attribute24', 'attribute25', 'attribute26', 'attribute27', 'attribute28', 'attribute29', 'attribute30', 'attribute31', 'attribute32', 'attribute33', 'attribute34', 'attribute35', 'attribute36', 'attribute37', 'attribute38', 'attribute39', 'attribute40', 'attribute41', 'attribute42', 'attribute43', 'attribute44', 'attribute45', 'attribute46', 'attribute47', 'attribute48', 'attribute49', 'attribute50', 'attribute51', 'attribute52', 'attribute53', 'attribute54', 'attribute55', 'attribute56', 'attribute57', 'attribute58', 'attribute59', 'attribute60', 'attribute61', 'attribute62', 'attribute63', 'attribute64', 'attribute65', 'attribute66', 'attribute68', 'attribute69', 'att

NameError: name 'spearmanr' is not defined

In [7]:
import numpy as np
subgroups = results_dict['PCA']['Depth-First Search']['results_org']
df = dfs_dict['PCA']

In [29]:
features = [feature for feature in df.columns if feature != 'target']
centroids = []
subgroup_cohesion = []
for subgroup_index in range(len(subgroups)):
    oper = str(subgroups["subgroup"][subgroup_index])
    oper = oper.replace("AND", "&")
    if oper.find(":") >= 0 :
        newOpers = []
        splitOper = oper.split(" & ")
        for j in range(len(splitOper)-1, -1, -1) :
            dpIndex = splitOper[j].find(":")
            if dpIndex >= 0 :
                attr = splitOper[j][:dpIndex]
                brIndex = splitOper[j].find("[")
                dpIndex2 = splitOper[j].find(":", dpIndex+1)
                lb = splitOper[j][brIndex+1:dpIndex2]
                ub = splitOper[j][dpIndex2+1:-1]
                newOpers.append(attr+">="+lb)
                newOpers.append(attr+"<="+ub)
                del splitOper[j]
        splitOper += newOpers
        oper = " & ".join(splitOper)
        print(df.columns)
        print(oper)
    subgroup_df = df[df.eval(oper)][features].astype(float)
    # subgroup_df = df[df.eval(str(subgroups.loc[subgroup_index, "subgroup"]).replace('AND', 'and'))].astype(float)
    centroid = subgroup_df.mean()
    centroids.append(centroid)
    avg_distance_to_centroid = np.linalg.norm(subgroup_df-centroid, axis=1).mean() 
    subgroup_cohesion.append(avg_distance_to_centroid)

Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC1>=2.85 & PC1<=3.42
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC3>=-1.12 & PC3<=-0.33
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC4>=-0.62 & PC4<=0.26
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC2>=-0.49 & PC2<=-0.08
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC2>=2.25 & PC1>=-4.26 & PC1<=-0.51
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC3<-1.57 & PC1>=-4.26 & PC1<=-0.51
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC1>=3.42 & PC3>=-1.12 & PC3<=-0.33
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC4>=0.26 & PC4<=1.56
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC3>=-1.57 & PC3<=-1.12 & PC1>=2.85 & PC1<=3.42
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object')
PC3>=-1.12 & PC3<=-0.33 & PC2>=-0.49 & PC2<=-0.08
Index(['target', 'PC1', 'PC2', 'PC3', 'PC4'], dtype='object

UndefinedVariableError: name 'Dataset' is not defined

In [31]:
subgroups.iloc[60:70,]

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift
60,0.002007,PC2: [-0.08:2.25[ AND PC4<-1.92,3,266,2,130,263,0.011278,0.988722,0.015385,0.984615,0.666667,0.486692,0.488722,1.364103
61,0.002007,PC1: [-0.51:2.85[ AND PC4: [-0.62:0.26[,3,266,2,130,263,0.011278,0.988722,0.015385,0.984615,0.666667,0.486692,0.488722,1.364103
62,0.001922,PC1: [2.85:3.42[ AND PC2<-2.01,1,266,1,130,265,0.003759,0.996241,0.007692,0.992308,1.000000,0.486792,0.488722,2.046154
63,0.000339,PC2: [-0.08:2.25[ AND PC4: [0.26:1.56[,8,266,4,130,258,0.030075,0.969925,0.030769,0.969231,0.500000,0.488372,0.488722,1.023077
64,0.000170,PC3<-1.57 AND PC4: [-0.62:0.26[,4,266,2,130,262,0.015038,0.984962,0.015385,0.984615,0.500000,0.488550,0.488722,1.023077
65,0.000085,PC2: [-0.08:2.25[ AND PC3: [-0.33:2.52[,2,266,1,130,264,0.007519,0.992481,0.007692,0.992308,0.500000,0.488636,0.488722,1.023077
66,0.000000,Dataset,266,266,130,130,0,1.000000,0.000000,1.000000,0.000000,0.488722,NaN,0.488722,1.000000
67,-0.001074,PC2>=2.25 AND PC4: [-1.92:-0.62[,19,266,9,130,247,0.071429,0.928571,0.069231,0.930769,0.473684,0.489879,0.488722,0.969231
68,-0.001668,PC2: [-0.49:-0.08[ AND PC3<-1.57,5,266,2,130,261,0.018797,0.981203,0.015385,0.984615,0.400000,0.490421,0.488722,0.818462
69,-0.001837,PC2<-2.01 AND PC4: [-1.92:-0.62[,1,266,0,130,265,0.003759,0.996241,0.000000,1.000000,0.000000,0.490566,0.488722,0.000000


In [28]:
oper

'PC3>=-0.33 & PC3<=2.52 & PC2>=-0.08 & PC2<=2.25'

In [22]:
for i in loaded_dict['vanilla']:
    print(i)

Best-First Search
Depth-First Search
Apriori
Beam Search


In [11]:
df_en = pd.DataFrame(loaded_dict)

ValueError: If using all scalar values, you must pass an index